In [1]:
PROJECT_NAME = "wheel_axle_phi"

In [2]:
from langchain_community.chat_models import ChatOllama 
from langchain import LLMChain, PromptTemplate
from langchain.memory import ConversationBufferWindowMemory

In [3]:
template = """You are a helpful assistant for creating a .sif file for use in multiphysics simulations with the ELMER software. Analyze the previous geometry and the simulation requirements the user wants, and ask questions on all required sections explained below until you have all the data to produce the file. When writing the file follow these sections meticulously:

Header Section: (Automatically include this section in the .sif file.)
Header
CHECK KEYWORDS Warn
Mesh DB "." "."
Include Path ""
Results Directory ""
End

Simulation Details:

Specify if the simulation type is "Steady state" or "Transient".
Constants Section: (Automatically include this section.)
Constants
...
End

Body Properties: (Refer to the physical groups defined in the .geo file.)
Body 1
...
End

Solver Configuration: (Automatically include this section.)
Solver 1
...
End

Equation Settings: (Automatically include this section.)
Equation 1
...
End

Material Properties:

Reference the named physical groups from the .geo file for material assignments.
Initial Conditions: (Automatically include this section.)
Initial Condition 1
...
End

Boundary Conditions:

Based on the geometry defined in the .geo file, specify boundary conditions for each named physical group.
Output Instructions:

Produce only the final text of the .sif file.
Enclose the entire .sif file content within specific markers:
Start the file content with //BEGIN_SIF.
End the file content with //END_SIF.
Ensure that the text between these markers is exactly as it should appear in the .sif file, with correct syntax and formatting.
Example:
//BEGIN_SIF
[Insert the complete and final .sif file content here]
//END_SIF"

I provide an example of sif file for linear elesticity from the ELMER tutorials
!ElmerSolver input file from ElmerFront
!Saved        = Wed May 14 09:51:11 2003 User=apursula Host=hiisi.csc.fi
!Case         = Beam  
!Model dir    = /mnt/hiisi/wrk/apursula/tutorialfiles/elast_beam/linear
!Include path = 
!Results dir  = /mnt/hiisi/wrk/apursula/tutorialfiles/elast_beam/linear

!Bodies 1
!Equations 1
!Solvers 1
!Materials 1
!Body Forces 0
!Initial Conditions 0
!Boundary Conditions 2
!Boundaries 4

!echo on

Header
  CHECK KEYWORDS Warn
  Mesh DB "MESHDIR" "mesh1"
  Include Path ""
  Results Directory ""
End

Simulation
  Min Output Level = 0
  Max Output Level = 31
  Output Caller = True

  Coordinate System = "Cartesian 2D"
  Coordinate Mapping(3) = 1 2 3

  Simulation Type = "Steady State"
  Steady State Max Iterations = 20
  Output Intervals = 1

  Solver Input File = "Beam.sif"
  Output File = "Beam.dat"
  Post File = "Beam.ep"
  Mesh Input File
    File "Beam.mif"

End

Constants
  Gravity(4) = 0 -1 0 9.82
  Stefan Boltzmann = 5.67e-08
End

Body 1
  Name = "Body1"

  Equation = 1
  Material = 1
End

Equation 1
  Name = "Equation1"

  Stress Analysis = True
  Calculate Stresses = Logical False
End

Solver 1
  Exec Solver = "Always"
  Equation = "Stress Analysis"
  Variable = "Displacement"
  Variable Dofs = 2
  Linear System Solver = "Iterative"
  Linear System Iterative Method = "BiCGStab"
  Linear System Max Iterations = 300
  Linear System Convergence Tolerance = 1.0e-08
  Linear System Abort Not Converged = True
  Linear System Preconditioning = "ILU0"
  Linear System Residual Output = 1
  Steady State Convergence Tolerance = 1.0e-05
  Nonlinear System Convergence Tolerance = 1.0e-05
  Nonlinear System Max Iterations = 1
  Nonlinear System Newton After Iterations = 3
  Nonlinear System Newton After Tolerance = 1.0e-02
  Nonlinear System Relaxation Factor = 1.0
  Adaptive Error Limit = 0.1
  Adaptive Max Change = 2
  Adaptive Coarsening = Logical True
  Time Derivative Order = 2
End

Material 1
  Name = "Material1"

  Youngs Modulus = 200e9
  Poisson Ratio = 0.3
End

Boundary Condition 1
  Name = "Constraint1"
  Target Boundaries(1) = 4 

  Displacement 1 = 0
  Displacement 2 = 0
End


Boundary Condition 2
  Name = "Constraint2"
  Target Boundaries(1) = 3 

  Force 2 = Variable Coordinate 1
    Real
      0 0
      1 -1.0000e+07
    End
End

!End Of File

Here another example
Header
  CHECK KEYWORDS Warn
  Mesh DB "." "."
  Include Path ""
  Results Directory ""
End

Simulation
  Max Output Level = 1
  Coordinate System = Cartesian
  Coordinate Mapping(3) = 1 2 3
  Simulation Type = Steady state
  Steady State Max Iterations = 1
  Output Intervals(1) = 1
  Solver Input File = case.sif
  Post File = case.vtu
End

Constants
  Gravity(4) = 0 -1 0 9.82
  Stefan Boltzmann = 5.670374419e-08
  Permittivity of Vacuum = 8.85418781e-12
  Permeability of Vacuum = 1.25663706e-6
  Boltzmann Constant = 1.380649e-23
  Unit Charge = 1.6021766e-19
End

Body 1
  Target Bodies(1) = 1
  Name = "Body Property 1"
  Equation = 1
  Material = 1
  Initial condition = 1
End

Solver 1
  Equation = Linear elasticity
  Procedure = "StressSolve" "StressSolver"
  Exec Solver = Always
  Stabilize = True
  Optimize Bandwidth = True
  Steady State Convergence Tolerance = 1.0e-5
  Nonlinear System Convergence Tolerance = 1.0e-7
  Nonlinear System Max Iterations = 20
  Nonlinear System Newton After Iterations = 3
  Nonlinear System Newton After Tolerance = 1.0e-3
  Nonlinear System Relaxation Factor = 1
  Linear System Solver = Iterative
  Linear System Iterative Method = BiCGStab
  Linear System Max Iterations = 500
  Linear System Convergence Tolerance = 1.0e-10
  BiCGstabl polynomial degree = 2
  Linear System Preconditioning = ILU0
  Linear System ILUT Tolerance = 1.0e-3
  Linear System Abort Not Converged = False
  Linear System Residual Output = 10
  Linear System Precondition Recompute = 1
End

Equation 1
  Name = "elasticity"
  Active Solvers(1) = 1
End

Material 1
  Name = "Steel (stainless - generic)"
  Heat expansion Coefficient = 14.9e-6
  Youngs modulus = 200.0e9
  Density = 7925.0
  Heat Conductivity = 24.0
  Mesh Poisson ratio = 0.285
  Heat Capacity = 460.0
  Sound speed = 5100.0
  Poisson ratio = 0.3
End

Initial Condition 1
  Name = "InitialCondition"
  Displacement 1 = 0
  Displacement Velocity 1 = 0
  Displacement Velocity 2 = 0
  Displacement Velocity 3 = 0
  Displacement 3 = 0
  Displacement 2 = 0
End

Boundary Condition 1
  Target Boundaries(1) = 2 
  Name = "fixed"
  Displacement 1 = 0
  Displacement 2 = 0
  Displacement 3 = 0
End

Boundary Condition 2
  Target Boundaries(1) = 1 
  Name = "subjected_to_force"
  Force 2 = 200000000
  Force 1 = 0
  Force 3 = 0
End

There is no Output section in the sif file

If unclear, ask which surfaces correspond to a specific boundary condition.
To help you understanding the geometry of the system, below you will find the text of the .geo file with geometry. 
geo file:





// Switch to OpenCASCADE geometry kernel
SetFactory("OpenCASCADE");

// Define the dimensions
Rw = 5; // Radius of the wheel in cm
Ww = 2; // Width of the wheel in cm
Ra = 1; // Radius of the axle in cm
La = 20; // Length of the axle in cm

// Define the axle cylinder
// The axle is centered at the origin and spans from -La/2 to La/2 along the z-axis
Cylinder(1) = {{0, 0, -La/2, 0, 0, La, Ra}};

// Define the left wheel cylinder
// The left wheel is positioned such that its inner face is flush with the left end of the axle
Cylinder(2) = {{0, 0, -La/2 - Ww, 0, 0, Ww, Rw}};

// Define the right wheel cylinder
// The right wheel is positioned such that its inner face is flush with the right end of the axle
Cylinder(3) = {{0, 0, La/2, 0, 0, Ww, Rw}};

// Define physical groups for the axle and wheels
//Physical Volume("Axle") = {{1}};
//Physical Volume("LeftWheel") = {{2}};
//Physical Volume("RightWheel") = {{3}};//+

Physical Volume("MainBody") = {{1}};

Physical Surface("Top") = {{7}};
Physical Surface("Bottom") = {{2}};


{history}
Human: {human_input}
Assistant:"""

prompt = PromptTemplate(input_variables=["history", "human_input"], template=template)

In [4]:
#Saves .geo file from response
def extract_and_save_sif_file(response_text): 
    file_path = f"{PROJECT_NAME}.sif"
    try:
        # Use string formatting explained in system_geo.txt 
        start_marker = "//BEGIN_SIF"
        end_marker = f"//END_SIF"

        start_index = response_text.index(start_marker) + len(start_marker)
        end_index = response_text.index(end_marker, start_index)
        geo_content = response_text[start_index:end_index].strip()

        with open(file_path, 'w') as file:
            file.write(geo_content)
        print(f"{file_path} file saved")

    except ValueError as e:
        print(f"ERROR: .geo file NOT SAVED.")
        
        
        
        
import os

def generate_ELMER_mesh():
    try:
        current_directory = os.getcwd()
        msh_file = os.path.join(current_directory, f'{PROJECT_NAME}.msh')  
        !"C:\Program Files\ElmerFEM-nogui-nompi-Windows-AMD64\bin\ElmerGrid.exe" 14 2 "{msh_file}" -autoclean
        print(f"ELMER project created in {PROJECT_NAME}")
        print(f"Copy .sif file in {PROJECT_NAME}")
        
    except ValueError as e:
        print(f"ERROR.")

In [5]:
phi3 = ChatOllama(model="phi3:mini-128k")

/var/folders/20/5gmmnyxx36l0kvfhyprtl9b00000gn/T/ipykernel_64695/1406247478.py:1: LangChainDeprecationWarning: The class `ChatOllama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import ChatOllama``.
  phi3 = ChatOllama(model="phi3:mini-128k")


In [6]:
chain = LLMChain(
    llm=phi3,
    prompt=prompt,
    verbose=False,
    memory=ConversationBufferWindowMemory(k=8),
    llm_kwargs={"max_tokens":8192,"temperature": 0.00}
)

/var/folders/20/5gmmnyxx36l0kvfhyprtl9b00000gn/T/ipykernel_64695/4075912485.py:5: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory=ConversationBufferWindowMemory(k=8),
/var/folders/20/5gmmnyxx36l0kvfhyprtl9b00000gn/T/ipykernel_64695/4075912485.py:1: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  chain = LLMChain(


In [7]:
generate_ELMER_mesh()

zsh:1: command not found: C:\Program Files\ElmerFEM-nogui-nompi-Windows-AMD64\bin\ElmerGrid.exe
ELMER project created in wheel_axle_phi
Copy .sif file in wheel_axle_phi


In [8]:
output = chain.predict(
    human_input="""Create a sif file for a 'linear elasticity' simulation of the wheel and axle with the boundaries defined in the 
mesh file. Boundary Condition at the "Bottom" should be fixed (no deformation), while the Boudary Condition at the "Top" implies a force 
of 5000000000 N in the z direction""")
print(output)

 //BEGIN_SIF
Header
  CHECK KEYWORDS Warn
  Mesh DB ".", "mesh1.mif"
  Include Path ""
  Results Directory ""
End

Simulation
  Max Output Level = 31
  Coordinate System = Cartesian
  Coordinate Mapping(3) = 1 2 3
  Simulation Type = Steady state
  Steady State Max Iterations = 20
  Output Intervals(1) = 1
  Solver Input File = "linear_elasticity.sif"
  Post File = "linear_elasticity.ep"
End

Constants
  Gravity(4) = 0 -1 0 9.82
  Youngs Modulus = 200e9
  Poisson Ratio = 0.3
  Permittivity of Vacuum = 8.85418781e-12
  Permeability of Vacuum = 1.25663706e-6
  Stefan Boltzmann = 5.670374419e-08
  Unit Charge = 1.6021766e-19
End

Body 1
  Target Bodies(1) = 1
  Name = "Wheel and Axle"
  Equation = 1
  Material = 1
End

Solver 1
  Exec Solver = Always
  Procedure = "StressSolve" "StressSolver"
  Stabilize = True
  Optimize Bandwidth = True
  Steady State Convergence Tolerance = 1.0e-5
  Nonlinear System Convergence Tolerance = 1.0e-7
  Nonlinear System Max Iterations = 20
  Nonlinear Syste

In [9]:
output = chain.predict(
    human_input="""
1. Simulation Type: "Steady state"
2. Material Properties of MainBody: use typical values for steel
3. No body forces
4. Use default ones solver settings.
5. Displacement
Also, for Initial condition, assume displacement = 0
""")
print(output)

 //BEGIN_SIF
Header
  CHECK KEYWORDS Warn
  Mesh DB ".", "mesh1.mif"
  Include Path ""
  Results Directory ""
End

Simulation
  Max Output Level = 31
  Coordinate System = Cartesian
  Coordinate Mapping(3) = 1 2 3
  Simulation Type = Steady state
  Steady State Max Iterations = 20
  Output Intervals(1) = 1
  Solver Input File = "linear_elasticity.sif"
End

Constants
  Gravity(4) = 0 -1 0 9.82
  Youngs Modulus = 200e9 // typical value for steel
  Poisson Ratio = 0.3
  Permittivity of Vacuum = 8.85418781e-12
  Permeability of Vacuum = 1.25663706e-6
  Steady State Convergence Tolerance = 1.0e-5
  Nonlinear System Convergence Tolerance = 1.0e-7
  Nonlinear System Max Iterations = 20
  Nonlinear System Newton After Iterations = 3
  Nonlinear System Newton After Tolerance = 1.0e-3
  Nonlinear System Relaxation Factor = 1
  Linear System Solver = Iterative
  Linear System Iterative Method = BiCGStab
  Linear System Max Iterations = 500
  Linear System Convergence Tolerance = 1.0e-10
  Nonline

In [ ]:
extract_and_save_sif_file(output)

In [ ]:
import os
new_directory = os.path.join(os.getcwd(), f"{PROJECT_NAME}")
os.chdir(new_directory)

In [ ]:
!"C:\Program Files\ElmerFEM-nogui-nompi-Windows-AMD64\bin\ElmerSolver.exe" "INSERT_FILE_PATH_HERE"